In [ ]:
%pip install pinecone-client

In [ ]:
%pip install -U sentence-transformers
%pip install -U langchain

In [ ]:
%pip install python-dotenv

In [ ]:
%pip install tqdm

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

PINECONE_KEY = os.getenv('PINECONE_KEY')

In [3]:
from pinecone import Pinecone

pinecone = Pinecone(api_key=PINECONE_KEY)

c:\Users\Bruno\Documents\Universidad\TACC\tacc-pukyu-yachay\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


#Index Functions

In [4]:
from pinecone import ServerlessSpec

def create_or_use_index(index_name):
  """
  Checks if the given index exists and creates it if it doesn't.

  Args:
      index_name: The name of the Pinecone index.

  Returns:
      The Pinecone index.
  """
  if index_name in pinecone.list_indexes().names():
    index = pinecone.Index(index_name)
    print(f"Found existing index: {index_name}")
  else:
    print(f"Creating new index: {index_name}")
    index = pinecone.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
        )
  return index

def delete_index(index_name):
  """
  Deletes the given Pinecone index.

  Args:
      index_name: The name of the Pinecone index.
  """
  if index_name in pinecone.list_indexes().names():
    pinecone.delete_index(index_name)
    print(f"Deleted index: {index_name}")
  else:
    print(f"Index {index_name} does not exist.")

In [5]:
index_name = 'pukyu-recetas'

In [6]:
index = create_or_use_index(index_name)

Found existing index: pukyu-recetas


In [60]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 260}},
 'total_vector_count': 260}

#Documents Preproccessing

In [ ]:
%pip install chardet

In [44]:
import chardet
from typing import List, Tuple

def preprocess_text_files(folder_path: str) -> List[Tuple[str, str]]:
    """
    Lee y preprocesa archivos de texto de una carpeta, manejando diferentes codificaciones.

    Args:
    folder_path (str): Ruta a la carpeta que contiene los archivos de texto.

    Returns:
    List[Tuple[str, str]]: Una lista de tuplas, cada una conteniendo el nombre del archivo
    y su contenido como una cadena de texto.
    """
    processed_files = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            
            # Leer el archivo en modo binario para detectar la codificación
            with open(file_path, 'rb') as file:
                raw_data = file.read()
            
            # Detectar la codificación
            detected = chardet.detect(raw_data)
            encoding = detected['encoding']

            # Leer el archivo con la codificación detectada
            try:
                with open(file_path, 'r', encoding=encoding) as file:
                    content = file.read()
                
                # Realizar cualquier limpieza o normalización adicional aquí
                # Por ejemplo:
                content = content.replace('\r\n', '\n')  # Normalizar saltos de línea
                content = ' '.join(content.split())  # Eliminar espacios múltiples
                
                processed_files.append((filename, content))
                print(f"Procesado: {filename} (Codificación: {encoding})")
            except UnicodeDecodeError:
                print(f"Error al decodificar: {filename}. Intentando con UTF-8.")
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        content = file.read()
                    processed_files.append((filename, content))
                    print(f"Procesado con UTF-8: {filename}")
                except UnicodeDecodeError:
                    print(f"No se pudo procesar: {filename}")

    return processed_files

In [48]:
recepies_path = '../recetas_quy/'

preprocessed_files = preprocess_text_files(recepies_path)

Procesado: Cau_Cau_de_Pollo.txt (Codificación: utf-8)
Procesado: Cañitas_fritas_rellenas_de_crema.txt (Codificación: utf-8)
Procesado: Cebiche_de_Pollo.txt (Codificación: utf-8)
Procesado: Champiñones_al_ajillo.txt (Codificación: utf-8)
Procesado: Chanfainita.txt (Codificación: utf-8)
Procesado: Charquican.txt (Codificación: utf-8)
Procesado: Chicharrón_de_Pescado.txt (Codificación: utf-8)
Procesado: Chili_con_carne.txt (Codificación: utf-8)
Procesado: Chimichanga_dulce.txt (Codificación: utf-8)
Procesado: Chipirones_a_la_plancha.txt (Codificación: utf-8)
Procesado: Chipirones_encebollados.txt (Codificación: utf-8)
Procesado: Choquitos_en_su_tinta.txt (Codificación: utf-8)
Procesado: Chorizos_al_vino.txt (Codificación: utf-8)
Procesado: Chorizos_a_la_sidra.txt (Codificación: utf-8)
Procesado: Chuleta_de_cerdo_con_piperrada.txt (Codificación: utf-8)
Procesado: Chupe_de_Camaroncito_Chino.txt (Codificación: utf-8)
Procesado: Churros.txt (Codificación: utf-8)
Procesado: Cochifrito.txt (Cod

#Embeddings creation

In [47]:
import os
from typing import List, Tuple, Dict
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

def process_documents_and_create_embeddings(
    processed_documents: List[Tuple[str, str]],
    model_name: str = 'all-MiniLM-L6-v2',
    chunk_size: int = 100,
    chunk_overlap: int = 20
) -> Tuple[List[Dict[str, str]], List[List[float]]]:
    """
    Procesa documentos preprocesados, los divide en chunks y crea embeddings.

    Args:
    processed_documents (List[Tuple[str, str]]): Lista de tuplas (nombre_archivo, contenido).
    model_name (str): Nombre del modelo de SentenceTransformer a utilizar.
    chunk_size (int): Tamaño máximo de cada chunk de texto.
    chunk_overlap (int): Superposición entre chunks consecutivos.

    Returns:
    Tuple[List[Dict[str, str]], List[List[float]]]: Una tupla conteniendo la lista de chunks 
    (con metadatos) y la lista de sus embeddings correspondientes.
    """
    # Inicializar el modelo de embedding
    model = SentenceTransformer(model_name)

    # Inicializar el divisor de texto
    #text_splitter = RecursiveCharacterTextSplitter(
    #    chunk_size=chunk_size,
    #    chunk_overlap=chunk_overlap
    #)

    #all_chunks = []
    all_texts = []

    # Leer y procesar cada archivo en la carpeta
    for filename, content in processed_documents:
        #chunks = text_splitter.split_text(content)
        #for chunk in chunks:
        #    all_texts.append({
        #        "text": chunk,
        #        "source": filename
        #    })
        all_texts.append({
            "text": content,
            "source": filename
        })

    # Extraer solo el texto de los chunks para crear embeddings
    texts = [text["text"] for text in all_texts]

    # Crear embeddings para todos los chunks
    embeddings = model.encode(texts)

    return all_texts, embeddings

In [53]:
all_texts, embeddings = process_documents_and_create_embeddings(preprocessed_files)

#Uploading embeddings to Pinecone

In [31]:
import uuid

def upload_to_pinecone(
    texts: List[str],
    embeddings: List[List[float]],
    batch_size: int = 100
) -> None:
    """
    Sube chunks de texto y sus embeddings correspondientes a Pinecone.

    Args:
    chunks (List[str]): Lista de chunks de texto.
    embeddings (List[List[float]]): Lista de embeddings correspondientes a los chunks.
    batch_size (int): Tamaño del lote para las operaciones de upsert.

    Returns:
    None
    """
    # Verificar que el número de chunks y embeddings coincida
    if len(texts) != len(embeddings):
        raise ValueError("El número de chunks y embeddings debe ser el mismo.")

    # Preparar los datos para la subida
    total_vectors = len(texts)
    for i in range(0, total_vectors, batch_size):
        batch_chunks = texts[i:i+batch_size]
        batch_embeddings = embeddings[i:i+batch_size]
        
        # Crear IDs únicos para cada vector
        ids = [str(uuid.uuid4()) for _ in range(len(batch_chunks))]
        
        # Preparar los vectores para el upsert
        vectors_to_upsert = list(zip(ids, batch_embeddings, [{"text": chunk} for chunk in batch_chunks]))
        
        # Realizar el upsert
        index.upsert(vectors=vectors_to_upsert)
        
        print(f"Subidos {i+len(batch_chunks)} de {total_vectors} vectores")

    print("Subida completada.")

In [59]:
texts = [file['text'] for file in all_texts]
upload_to_pinecone(texts, embeddings)

Subidos 100 de 260 vectores
Subidos 200 de 260 vectores
Subidos 260 de 260 vectores
Subida completada.


#Testing the RAG

In [61]:
def simulate_llm_response(query: str, chunks: List[Dict[str, float]]) -> str:
    """
    Simula la respuesta de un LLM basada en la consulta y los chunks recuperados.
    
    En un escenario real, esta función sería reemplazada por una llamada al modelo llama2-7b-hf.
    """
    # Esta es una simulación muy básica
    combined_context = " ".join([chunk for chunk, _ in chunks])
    response = f"Basándome en la consulta '{query}' y el contexto proporcionado, "
    response += f"puedo decir que la información relevante incluye {len(chunks)} fragmentos de texto. "
    response += f"El contexto total tiene {len(combined_context)} caracteres. "
    response += "Una respuesta más detallada se generaría utilizando el modelo LLM real."
    return response

In [64]:
def test_rag(
    query: str,
    model_name: str = 'all-MiniLM-L6-v2',
    top_k: int = 5
) -> None:
    """
    Prueba el funcionamiento del RAG: recupera chunks relevantes y simula una respuesta.

    Args:
    query (str): La consulta del usuario.
    api_key (str): API key de Pinecone.
    environment (str): Entorno de Pinecone.
    index_name (str): Nombre del índice de Pinecone a utilizar.
    model_name (str): Nombre del modelo de SentenceTransformer a utilizar.
    top_k (int): Número de chunks más relevantes a recuperar.

    Returns:
    None
    """
    # Inicializar el modelo de embedding
    model = SentenceTransformer(model_name)

    # Crear el embedding de la consulta
    query_embedding = model.encode(query).tolist()

    # Realizar la búsqueda en Pinecone
    search_results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    # Extraer los chunks y sus puntuaciones
    retrieved_chunks = [
        (result.metadata['text'], result.score) 
        for result in search_results.matches
    ]

    print(f"Consulta: {query}\n")
    print("Chunks recuperados:")
    for i, (chunk, score) in enumerate(retrieved_chunks, 1):
        print(f"\nChunk {i} (Score: {score:.4f}):")
        print(chunk[:200] + "..." if len(chunk) > 200 else chunk)

    # Simular la generación de respuesta
    simulated_response = simulate_llm_response(query, retrieved_chunks)
    
    print("\nRespuesta simulada del LLM:")
    print(simulated_response)

In [68]:
test_query = "¿receta queso rawk asqa?"
test_rag(test_query)

Consulta: ¿receta queso rawk asqa?

Chunks recuperados:

Chunk 1 (Score: 0.4487):
receta marmitako bonito norte nisqa ingredientes qanchis pachak pichqa chunka gramo musuq bonito kilogramo papa huk clavo ajo huk unidad ch’uñu huk unidad italiano verde pimienta huk kuartal unidad pu...

Chunk 2 (Score: 0.4252):
receta quri horno ingredientes quri kilo soqta pachak gramo papa kimsa unidad clavokuna ajo huk unidad ch’uñu huk phatma puka pimentu huk phatma verde pimienta huk unidad limón huk cuchara perejil pha...

Chunk 3 (Score: 0.4018):
receta espárrago tartar salsa ingredientes iskay qutu yuraq espárrago iskay pachak pichqa chunka gramo tartar salsa kimsa litro yaku iskay limón unidad huk cuchara kachi instrucciones ñawpaqta yanapay...

Chunk 4 (Score: 0.3994):
receta Ensalada americana ingredientes huk uma k’uyusqa col huk uma romaine lechuga iskay unidad zanahoria huk unidad puka ch’uñu pusaq camarón unidad pusaq unidad cangrejo k’aspikuna iskay unidad sin...

Chunk 5 (Score: 0.3992)